<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Content Based Filtering

Estimated time needed: **25** minutes

## Objectives

After completing this lab you will be able to:

-   Create a recommendation system using Content Based Filtering


Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the users. These systems have become ubiquitous and can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore Content-based recommendation systems and implement a simple version of one using Python and the Pandas library.


### Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Content-Based Filtering</a></li>
    </ol>
</div>
<br>


<a id="ref1"></a>

# Acquiring the Data


To acquire and extract the data, simply run the following Bash scripts:  
Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). Lets download the dataset. To download the data, we will use **`!wget`** to download it from IBM Object Storage.  
**Did you know?** When it comes to Machine Learning, you will likely be working with large datasets. As a business, where can you host your data? IBM is offering a unique opportunity for businesses, with 10 Tb of IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)


In [1]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2020-11-03 16:10:05--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  20.4MB/s    in 7.8s    

2020-11-03 16:10:13 (19.6 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


Now you're ready to start working with the data!


<a id="ref2"></a>

# Preprocessing


First, let's get all of the imports out of the way:


In [2]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that (less weight to compute)
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:


In [3]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies_df.shape

(34208, 3)

Let's also remove the year from the **title** column by using pandas' **replace** function and store it in a new **year** column.


In [5]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
#by using parentheses, a movie called 1984 won´t be extracted because its name does not comply with the extracting rule '(\(\d\d\d\d\))'
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)

#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)

#Removing the years from the 'title' column and replacing it with an empty space''.
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')

#Applying the strip function to get rid of any ending whitespace characters that may have appeared
#lambda is an anonymous function that refers to x which wull be subject to the next command (strip function)
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [6]:
type(movies_df['genres'])

pandas.core.series.Series

With that, let's also split the values in the **Genres** column into a **list of Genres** to simplify future use. This can be achieved by applying Python's split string function on the correct column.


In [7]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


Note:\
</p> 1) movies_df['genres']= means that the whole column will storage data\
</p> 2) movies_df.genres.str.split , means that every data on the column will be subject to a function.

In [8]:
type(movies_df['genres'])

pandas.core.series.Series

Since keeping genres in a **list** format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the **list of genres** to a **vector** where each column corresponds to one possible value of the feature. This encoding is needed for feeding(switching from) categorical data. In this case, we store every different genre in columns that contain either 0 or 1 where 1 shows that a movie encompasses that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable (another dataframe) since genres won't be important for our first recommendation system.

In [9]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
moviesWithGenres_df.shape

(34208, 24)

In [11]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


Next, let's look at the ratings dataframe.


In [12]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [13]:
ratings_df.shape

(22884377, 4)

Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.


In [14]:
#Drop removes a specified row or column from a dataframe (drop labels from the index (0 or ‘index’) or columns (1 or ‘columns’))
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [15]:
ratings_df.shape

(22884377, 3)

<a id="ref3"></a>

# Content-Based Recommendation System


Now, let's take a look at how to implement **Content-Based** or **Item-Item Recommendation Systems**. This technique attempts to figure out what a user's favourite aspects of an item are, and then recommend similar items that present those aspects. In our case, we're going to try to figure out the input's favorite genres from the movies and ratings given.

Let's begin by creating an "input user" (someone who visited the "movies-site") to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the **userInput**. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [29]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
print("Type of Data being inserted into WebSite Rankings is", type(userInput))
print()
inputMovies = pd.DataFrame(userInput)
inputMovies

Type of Data being inserted into WebSite Rankings is <class 'list'>



,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


#### Add movieId to input user (to inputMovies actually)

With the input complete, let's extract the input movie's ID's from the **movies_df** dataframe and add them into **inputMovies**.

We can achieve this by first filtering out the rows that contain the input movie's title from **movies_df** and then merging this subset into **inputMovies**. We also drop unnecessary columns for the input to save memory space.

It is pretty much a "procv" (portuguese excel) function.


**Alex´s Special Extra Effort 1 =>** In order to understand this lenghty and counterintuitive sequence of commands, let me try to delve into each one of them in detail:

In [30]:
A = (inputMovies['title'].tolist())
print("inputMovies['title'].tolist() is => ", A)
print()
print("Data Type for A is =>", type(A))

inputMovies['title'].tolist() is =>  ['Breakfast Club, The', 'Toy Story', 'Jumanji', 'Pulp Fiction', 'Akira']

Data Type for A is => <class 'list'>


In [31]:
B = (movies_df['title'].isin(inputMovies['title'].tolist()))
print("The Check B (movies_df['title'].isin(inputMovies['title'].tolist())) => ")
print(B)
print()
print("This is the Procv function which assigns 'True' to the matches and 'False' to non-matches")

The Check B (movies_df['title'].isin(inputMovies['title'].tolist())) => 
0         True
1         True
2        False
3        False
4        False
         ...  
34203    False
34204    False
34205    False
34206    False
34207    False
Name: title, Length: 34208, dtype: bool

This is the Procv function which assigns 'True' to the matches and 'False' to non-matches


In [32]:
C = movies_df[B]
print(C)
print()
print("The Dimension of C (inputId) is =>", C.shape)
print()
print(type(C))
print()
print("C is inputId")

      movieId                title  \
0           1            Toy Story   
1           2              Jumanji   
293       296         Pulp Fiction   
1246     1274                Akira   
1885     1968  Breakfast Club, The   

                                                 genres  year  
0     [Adventure, Animation, Children, Comedy, Fantasy]  1995  
1                        [Adventure, Children, Fantasy]  1995  
293                    [Comedy, Crime, Drama, Thriller]  1994  
1246             [Action, Adventure, Animation, Sci-Fi]  1988  
1885                                    [Comedy, Drama]  1985  

The Dimension of C (inputId) is => (5, 4)

<class 'pandas.core.frame.DataFrame'>

C is inputId


After these 3 steps, we know which rows in **movies_df** dataframe are the Movies that the user watched and rated plus their correspondent movieId. The User does not know movieId, only the movie titles. Now we need to "blend" two sources of information: one that contains movieId and the other that contains ratings.

**Note**: C = movie_df[B] is not the **movie_df** dataframe ifself, it is sort of a "new and small" df whose content was extracted from the BIG dataframe for the rows assigned boolean value **True** by the functions (tolist) and (isin) in this order.

In [34]:
inputMovies = pd.merge(C, inputMovies)
print(inputMovies)
print()
print("Dimensions for inputMovies => ", inputMovies.shape)
print()

   movieId                title  \
0        1            Toy Story   
1        2              Jumanji   
2      296         Pulp Fiction   
3     1274                Akira   
4     1968  Breakfast Club, The   

                                              genres  year  rating  
0  [Adventure, Animation, Children, Comedy, Fantasy]  1995     3.5  
1                     [Adventure, Children, Fantasy]  1995     2.0  
2                   [Comedy, Crime, Drama, Thriller]  1994     5.0  
3             [Action, Adventure, Animation, Sci-Fi]  1988     4.5  
4                                    [Comedy, Drama]  1985     5.0  

Dimensions for inputMovies =>  (5, 5)



This last step added inputMovies['rating'] to inputId dataframe

In [35]:
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)   # 1 for 'column'
print("Final Product is a connection between two formerly independent dataframes")
print()
print(inputMovies)
print()

Final Product is a connection between two formerly independent dataframes

   movieId                title  rating
0        1            Toy Story     3.5
1        2              Jumanji     2.0
2      296         Pulp Fiction     5.0
3     1274                Akira     4.5
4     1968  Breakfast Club, The     5.0



In [36]:
#Filtering out the movies by title (it is like a 'procv')
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


So far, we have User´s ratings linked to movies identification index for the BIG dataframe (movies_df).\
Now, we have to learn User's preferences, so let's extract a subset of movies from **movies_df** that will display all movies the User has watched and rated.\
This new subset will bring the One Hot Encoding for genres which will provide us a base to build User´s preferences (or cinema tastes).

**Alex´s Special Extra Effort 2 =>** In order to understand this lenghty and counterintuitive sequence of commands, let me try to delve into each one of them in detail:

In [37]:
eins = (inputMovies['movieId'].tolist())
print("The IDs that we need to extract from 'movies_df' is:")
print(eins)

The IDs that we need to extract from 'movies_df' is:
[1, 2, 296, 1274, 1968]


In [40]:
zwei = (moviesWithGenres_df['movieId'].isin(eins))
print("The Procv-like Vector is:")
print(zwei)

The Procv-like Vector is:
0         True
1         True
2        False
3        False
4        False
         ...  
34203    False
34204    False
34205    False
34206    False
34207    False
Name: movieId, Length: 34208, dtype: bool


In [42]:
drei = moviesWithGenres_df[zwei]
print("Finally, we have the ratings´ categories (genres) associated to User´s movies")
print()
print(drei)
print()

Finally, we have the ratings´ categories (genres) associated to User´s movies

      movieId                title  \
0           1            Toy Story   
1           2              Jumanji   
293       296         Pulp Fiction   
1246     1274                Akira   
1885     1968  Breakfast Club, The   

                                                 genres  year  Adventure  \
0     [Adventure, Animation, Children, Comedy, Fantasy]  1995        1.0   
1                        [Adventure, Children, Fantasy]  1995        1.0   
293                    [Comedy, Crime, Drama, Thriller]  1994        0.0   
1246             [Action, Adventure, Animation, Sci-Fi]  1988        1.0   
1885                                    [Comedy, Drama]  1985        0.0   

      Animation  Children  Comedy  Fantasy  Romance  ...  Horror  Mystery  \
0           1.0       1.0     1.0      1.0      0.0  ...     0.0      0.0   
1           0.0       1.0     0.0      1.0      0.0  ...     0.0      0.0   
293 

In [44]:
#Filtering out the movies from the input
userMovies_genreTable = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies_genreTable

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We'll only need the actual genre table, so let's clean this up a bit by:\
a) resetting the index and\
b) dropping the movieId, title, genres and year columns.


In [45]:
#Resetting the index to avoid future issues
userMovies_genreTable = userMovies_genreTable.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies_genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we're ready to start learning the input's preferences!\

But, before doing it, let´s remind ourselves that this sequence of reasoning is exactly the same we watched on previous video. However, on that video, the User had watched only 3 movies from a "Universe" of 6. Next step will actually connect User´s preferences-tastes to a broader table in order to create his-her personnal table of recommendation (or, maybe, choice...)

To do this, we're going to turn each genre into weights (absolute or relative numbers). We can do this by using the input's reviews and multiplying them into the input's genre table **(userGenreTable)** and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling Pandas's "dot" function.


In [46]:
inputMovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

In [47]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [55]:
userProfile.shape

(20,)

Now, we have the weights for every genre from user's preferences standpoint.\
This is known as the User Profile and by using it, we can recommend movies that satisfy the user's preferences.

Let's start by extracting the genre table from the original dataframe:\

From the class (video), this step corresponds to the second *vector x matrix* "multiplication" which is followed by a row-by-row adding procedure in order to assign a value (rating) to every movie on the "library".

In [52]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head(10)

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
genreTable.shape

(34208, 20)

With the input's profile **(User Preferences)** and the complete list of movies and their genres in hand **(from modified moviesWithGenres_df)**, we're going to take the weighted average (row-by-row adding) of every movie based on the input profile and recommend the top twenty movies that most satisfy it.


In [61]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
print()
print("The recommendationTable_df will look like")
print()
print(recommendationTable_df.head(10))
print()
print(recommendationTable_df.tail())


The recommendationTable_df will look like

movieId
1     0.594406
2     0.293706
3     0.188811
4     0.328671
5     0.188811
6     0.202797
7     0.188811
8     0.216783
9     0.062937
10    0.272727
dtype: float64

movieId
151697    0.069930
151701    0.000000
151703    0.139860
151709    0.202797
151711    0.000000
dtype: float64


In [58]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head(10)

movieId
5018      0.748252
26093     0.734266
27344     0.720280
148775    0.685315
6902      0.678322
117646    0.678322
64645     0.671329
81132     0.671329
122787    0.671329
2987      0.664336
dtype: float64

Now here's the recommendation table!


In [59]:
#The final recommendation table
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
664,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
1824,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2902,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
4923,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
6793,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
8605,26093,"Wonderful World of the Brothers Grimm, The","[Adventure, Animation, Children, Comedy, Drama...",1962
8783,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
9296,27344,Revolutionary Girl Utena: Adolescence of Utena...,"[Action, Adventure, Animation, Comedy, Drama, ...",1999
9825,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
11716,51632,Atlantis: Milo's Return,"[Action, Adventure, Animation, Children, Comed...",2003


In [60]:
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())].shape

(20, 4)

Final Command brings "loc" (664, 1824, etc) only. All other data are from 'movies_df' structure.

**CONCLUSION:** Every step that was performed on this Lab Session mirrors what was explained on the video-class. Coding here is a very important issue that was not covered on video though.

### Advantages and Disadvantages of Content-Based Filtering

##### Advantages

-   Learns user's preferences
-   Highly personalized for the user

##### Disadvantages

-   Doesn't take into account what others think of the item, so low quality item recommendations might happen
-   Extracting data is not always intuitive
-   Determining what characteristics of the item the user dislikes or likes is not always obvious


<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="https://www.ibm.com/analytics/spss-statistics-software">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://www.ibm.com/cloud/watson-studio">Watson Studio</a>


### Thank you for completing this lab!

## Author

Saeed Aghabozorgi

### Other Contributors

<a href="https://www.linkedin.com/in/joseph-s-50398b136/" target="_blank">Joseph Santarcangelo</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2020-08-27        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |
|                   |         |            |                                    |
|                   |         |            |                                    |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
